In [1]:
import pandas as pd
import numpy as np


In [2]:
h_waste = pd.read_csv("../data/Hazardous WasteIGD_EF_RCRA.csv", low_memory=False)
toxic_release = pd.read_csv("../data/Toxic ReleasesIGD_EF_TRI.csv")
tn_pop = pd.read_csv("../data/tn_county_pop.csv")
swm_permits = pd.read_csv("../data/SWM Permits.csv")
rare_s = pd.read_csv("../data/Rare Species by County.csv", encoding='windows-1252')
#streams = pd.read_csv("../data/Exceptional TN Streams.csv", encoding='windows-1252')
rare_e = pd.read_excel("../data/Exceptional TN Streams filtered.xlsx", sheet_name='Endangered')
oil_gas = pd.read_csv("../data/Oil and Gas Well Permits.csv")


In [3]:
h_waste.rename(columns={'county_name': 'county'}, inplace=True)
toxic_release.rename(columns={'county_name': 'county'}, inplace=True)
swm_permits.rename(columns={'County': 'county'}, inplace=True)
rare_s.rename(columns={'County': 'county'}, inplace=True)
rare_e.rename(columns={'County': 'county'}, inplace=True)
oil_gas.rename(columns={'County': 'county'}, inplace=True)


In [4]:
h_waste['county'] = h_waste['county'].str.upper()
tn_pop['county'] = tn_pop['county'].str.upper()
rare_s['county'] = rare_s['county'].str.upper()
rare_e['county'] = rare_e['county'].str.upper()
oil_gas['county'] = oil_gas['county'].str.upper()
swm_permits['county'] = swm_permits['county'].str.upper()


In [5]:
tn_waste = h_waste[h_waste['state_code']=='TN']
tn_toxic = toxic_release[toxic_release['state_code']=='TN']
tn_waste_center = swm_permits[(swm_permits['Permit Type']=='CONVENIENCE')&(swm_permits['Status']=='Active')]
oil_gas1 = oil_gas[oil_gas['Purpose of Well']!='Dry & Abandoned']
tp = tn_pop[['county','pop2020','pop2024' ,'growthSince2020', 'landArea' , 'density']]


In [6]:
tw = tn_waste.groupby('county').agg({'registry_id': 'size', 'city_name': pd.Series.nunique}).reset_index().sort_values('city_name', ascending=False)
tt = tn_toxic.groupby('county').agg({'registry_id': 'size', 'city_name': pd.Series.nunique}).reset_index().sort_values('city_name', ascending=False)
tcc = tn_waste_center.groupby('county').agg({'Permit Number': 'count', 'City': pd.Series.nunique}).reset_index().sort_values('City', ascending=False)
rare = rare_s.groupby('county').agg({'Type': 'size', 'Category': pd.Series.nunique}).reset_index().sort_values('Category', ascending=False)
rare_endangered = rare_e.groupby('county').agg({'HUC': 'size'}).reset_index().sort_values('HUC', ascending=False)
oilgas = oil_gas1.groupby('county').agg({'API No': 'size'}).reset_index().sort_values('API No', ascending=False)


In [7]:
tw.rename(columns={'registry_id': 'private_waste_loc_count', 'city_name': 'private_waste_city_count'}, inplace=True)
tt.rename(columns={'registry_id': 'toxic_release_loc_count', 'city_name': 'toxic_release_city_count'}, inplace=True)
tcc.rename(columns={'Permit Number': 'tn_convenience_loc_count', 'City': 'tn_convenience_city_count'}, inplace=True)
rare.rename(columns={'Type': 'tn_rare_count', 'Category': 'tn_rare_category'}, inplace=True)
rare_endangered.rename(columns={'HUC': 'endangered'}, inplace=True)
oilgas.rename(columns={'API No': 'oil_gas'}, inplace=True)


In [8]:
cc_pop = pd.merge(tp, tcc, on='county', how='left')
cc_tt = pd.merge(cc_pop, tt, on='county', how='left')
cc_tt_tw = pd.merge(cc_tt, tw, on='county', how='left')
cc_tt_tw_rc = pd.merge(cc_tt_tw, rare, on='county', how='left')
cc_tt_tw_rc_re = pd.merge(cc_tt_tw_rc, rare_endangered, on='county', how='left')
merged = pd.merge(cc_tt_tw_rc_re, oilgas, on='county', how='left')


In [9]:
merged['conveniece_land_mile_ratio'] = merged['landArea'] / merged['tn_convenience_loc_count']
merged['conveniece_pop_ratio'] =  merged['pop2024'] / merged['tn_convenience_loc_count']


In [10]:
merged

,county,pop2020,pop2024,growthSince2020,landArea,density,tn_convenience_loc_count,tn_convenience_city_count,toxic_release_loc_count,toxic_release_city_count,private_waste_loc_count,private_waste_city_count,tn_rare_count,tn_rare_category,endangered,oil_gas,conveniece_land_mile_ratio,conveniece_pop_ratio
0,SHELBY,929763,903727,-0.028003,761,1187.551905,1.0,1.0,215.0,7.0,2669,32,26,8,1.0,5.0,761.000000,903727.000000
1,DAVIDSON,716059,717317,0.001757,503,1426.077535,5.0,1.0,140.0,8.0,2079,48,70,11,18.0,62.0,100.600000,143463.400000
2,KNOX,480516,505958,0.052947,508,995.980315,7.0,1.0,70.0,5.0,981,17,51,10,2.0,NaN,72.571429,72279.714286
3,HAMILTON,367251,385126,0.048672,542,710.564576,6.0,3.0,130.0,6.0,1164,31,67,11,9.0,1.0,90.333333,64187.666667
4,RUTHERFORD,343298,373520,0.088034,619,603.424879,15.0,4.0,60.0,4.0,380,15,84,11,18.0,56.0,41.266667,24901.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,HANCOCK,6627,7066,0.066244,222,31.828829,1.0,1.0,2.0,1.0,6,1,45,8,3.0,58.0,222.000000,7066.000000
91,MOORE,6470,6780,0.047913,129,52.558140,1.0,0.0,2.0,1.0,2,1,19,6,2.0,1.0,129.000000,6780.000000
92,VAN BUREN,6174,6556,0.061872,273,24.014652,7.0,4.0,1.0,1.0,2,1,91,13,NaN,5.0,39.000000,936.571429
93,LAKE,6995,6194,-0.114510,166,37.313253,1.0,1.0,1.0,1.0,8,2,19,5,1.0,NaN,166.000000,6194.000000


In [11]:
#merged.to_csv('merged_count.csv',index=False)